Se tiene un RDD con libros en donde cada registro es un texto. Se
pide obtener todos los anagramas de mas de 7 letras que puedan
encontrarse. El formato de salida debe ser una lista de listas en donde
cada lista tiene un conjunto de palabras que son anagramas. Ejemplo:
[[discounter,introduces,reductions],[percussion,supersonic]...]  
[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqthf114z4)

Voy a armar un RDD de prueba con dos articulos que contienen las palabras 'percussion' y 'supersonic'

In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import wikipedia as wp
wp.set_lang('en')
articulo1 = wp.page('F-35').content
articulo2 = wp.page('Cajón de rumba').content
rdd = sc.parallelize([articulo1,articulo2])

Obtenemos las palabras de cada articulo (sin chequeo alguno, de una manera muy naive), lo cual nos da un RDD de listas de palabras. Para convertirlo en un un RDD de palabras usamos `flatMap`.

In [ ]:
rdd = rdd.map(lambda x: x.replace("\n","").split(" ")) #tokenizer naive
rdd = rdd.flatMap(lambda x: x).filter(lambda x: len(x) > 7)

Los anagramas son palabras o frases que contienen las mismas letras. En este caso particularmente son palabras nada mas. Dos palabras que tienen las mismas letras coinciden letras y cantidad de veces que aparercen, entonces no serviria por ejemplo usar el set de letras de dos palabras para saber si son anagramas, ya que perdemos la informacion de cuantas veces aparece cada letra.  
Si ordenamos las letras de ambas palabras, esas dos listas conciden si (solo si) son anagramas.
Como podemos tener las palabras comenzando en mayusculas, pasamos todo a minuscula primero. Luego usamos la lista ordenada de letras para agrupar. Eso nos da un RDD de `<lista ordenada, iterable de palabras>`.

In [ ]:
rdd = rdd.map(lambda x: x.lower())
rdd = rdd.map(lambda x: (tuple(sorted(x)),x))

Hay que tener en cuenta que la misma palabra puede aparecer varias veces en un mismo articulo o entre todos. Hay que eliminar las repeticiones. La forma sencilla con python es convertir ese iterable a un `set`.

In [ ]:
rdd = rdd.groupByKey().mapValues(set)

Filtramos para obtener los sets donde hay mas de una palabra. Luego convertimos los sets a listas segun se pidio.

In [ ]:
rdd = rdd.filter(lambda x: len(x[1]) > 1)
rdd = rdd.map(lambda x: list(x[1]))
rdd.collect()